In [7]:
#添加qid
with open('pred_dev.sql', 'r',encoding='utf-8') as f:
    lines = f.readlines()

with open('pred_dev.sql', 'w',encoding='utf-8') as f:
    for i, line in enumerate(lines, start=1):
        f.write(f"qid{i}\t{line}")


In [9]:
#删掉末尾的分号
with open('cleaned_pred_dev.sql', 'r',encoding='utf-8') as f:
    lines = f.readlines()
result = ''
with open('cleaned_pred_dev.sql', 'w',encoding='utf-8') as f:
    for line in lines:
        line = line.replace(';',' ')
        f.write(line)



In [10]:
#清洗codellama的口语化输出
import re

def extract_sql(content):
    # 提取SQL语句
    sqls = re.findall(r'```(.*?)```', content, re.DOTALL)
    if not sqls:
        return None
    # 保留最长的SQL语句
    sql = max(sqls, key=len)
    # 删除"SELECT"前面的所有词
    sql = re.sub(r'^.*?(SELECT)', r'\1', sql, flags=re.IGNORECASE)
    return sql.strip()

def process_file(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        qid, content = line.strip().split('\t', 1)
        if content.startswith('SELECT'):
            new_lines.append(line)
        else:
            sql = extract_sql(content)
            if sql is not None:
                new_lines.append(f'{qid}\t{sql}\n')
            else:
                new_lines.append(line)

    with open(filename, 'w',encoding='utf-8') as f:
        f.writelines(new_lines)

process_file('pred_dev.sql')


In [8]:
#清洗思源的html输出
import re
from bs4 import BeautifulSoup

def extract_sql(content):
    # 如果content以SELECT开头，直接返回
    if content.strip().startswith('SELECT'):
        return content.strip()
    
    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()
    
    # 从字符串中截取SQL语句
    match = re.search(r'(SELECT.*?;)', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # 如果截取不到SQL语句，返回整个字符串
    return text.strip()

with open('pred_dev.sql', 'r',encoding='utf-8') as f_in, open('cleaned_pred_dev.sql', 'w',encoding='utf-8') as f_out:
    for line in f_in:
        qid, content = line.rstrip('\n').split('\t', 1)
        cleaned_content = extract_sql(content)
        f_out.write(f'{qid}\t{cleaned_content}\n')
